In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import random  
import gc
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output
from torch.cuda import memory_allocated, empty_cache
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from glob import glob
from tqdm import tqdm

%matplotlib inline

- Yolov5 torch_버전 불러오기

In [2]:
# torch_ver Yolov5
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s',
                            device='cuda:0' if torch.cuda.is_available() else 'cpu')  # 예측 모델
yolo_model.classes = [0]  # 예측 클래스 (0 : 사람)

Using cache found in /Users/kimsungwook/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-4-17 Python-3.11.7 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


- 데이터 전처리 및 세부 조정

In [3]:
BATCH_SIZE = 40
EPOCH = 700
NUM_LAYERS = 1      # LSTM model: num_layers
start_dot = 11      # mp.solutions.pose 시작 포인트 (0: 얼굴부터 발목까지, 11: 어깨부터 발목까지)
n_CONFIDENCE = 0.3    # MediaPipe Min Detectin confidence check
y_CONFIDENCE = 0.3    # Yolv5 Min Detectin confidence check

mp_pose = mp.solutions.pose
attention_dot = [n for n in range(start_dot, 29)]

# 라인 그리기
if start_dot == 11:
    """몸 부분만"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24]]
    print('Pose : Only Body')

else:
    """얼굴 포함"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24], [9, 10], [0, 5], [0, 2], [5, 8], [2, 7]]
    print('Pose : Face + Body')

Pose : Only Body


In [4]:
# Yolov4 바운딩 box 안에서 media pipe 데이터 전처리 함수

def get_skeleton(video_path, attention_dot, draw_line):
    frame_length = 30 # LSTM 모델에 넣을 frame 수

    xy_list_list, xy_list_list_flip = [], []
    cv2.destroyAllWindows()
    pose = mp_pose.Pose(static_image_mode = True, model_complexity = 1, \
                        enable_segmentation = False, min_detection_confidence = n_CONFIDENCE)
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():

        while True:
            ret, img = cap.read()

            if ret == True:

                """ Yolo 바운딩 박스 및 좌표 추출"""
                img = cv2.resize(img, (640, 640))
                res = yolo_model(img)
                res_refine = res.pandas().xyxy[0].values
                nms_human = len(res_refine)
                if nms_human > 0:
                    for bbox in res_refine:
                        """바운딩 박스 상하좌우 크기 조절"""
                        xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
                        if xx1 < 0:
                            xx1 = 0
                        elif xx2 > 639:
                            xx2 = 639
                        if yy1 < 0:
                            yy1 = 0
                        elif yy2 > 639:
                            yy2 = 639

                        start_point = (xx1, yy1)
                        end_point = (xx2, yy2)

                        """ Yolov5 바운딩 박스 좌표 안에서 mediapipe Pose 추출"""
                        if bbox[4] > y_CONFIDENCE: # bbox[4] : confidence 데이터
                            # img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2) # 바운딩 박스 그리기 : 데이터 추출 확인용
                            c_img = img[yy1:yy2, xx1:xx2] # 바운딩 박스 좌표
                            results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolov5 바운딩 박스 좌표 안에서 'mp_pose' 좌표

                            if not results.pose_landmarks: continue
                            idx = 0
                            draw_line_dic = {}
                            xy_list, xy_list_flip = [], []
                            # 33 반복문 진행 : 33개 중 18개의 dot
                            for x_and_y in results.pose_landmarks.landmark:
                                if idx in attention_dot:
                                    xy_list.append(x_and_y.x)
                                    xy_list.append(x_and_y.y)
                                    xy_list_flip.append(1 - x_and_y.x)
                                    xy_list_flip.append(x_and_y.y)

                                    x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                                    draw_line_dic[idx] = [x, y]
                                idx += 1

                            if len(xy_list) != len(attention_dot) * 2:
                                print('Error : attention_dot 데이터 오류')

                            xy_list_list.append(xy_list)
                            xy_list_list_flip.append(xy_list_flip)

                            """mediapipe line 그리기 부분 : 데이터 추출(dot) 확인용"""
                            # for line in draw_line:
                            #     x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                            #     x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                            #     c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 4)
                            # # cv2.imshow('Landmark Image', img)
                            # cv2_imshow(img)
                            # cv2.waitKey(1)

            elif ret == False: break


        # 부족한 프레임 수 맞추기
        if len(xy_list_list_flip) < 15:
            return False, False
        elif len(xy_list_list_flip) < frame_length:
            f_ln = frame_length - len(xy_list_list_flip)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
                xy_list_list_flip.append(xy_list_list_flip[-1])

    cap.release()
    cv2.destroyAllWindows()


    return xy_list_list, xy_list_list_flip

In [5]:
video_path = '../../dataset_aihub/' # dataset 경로
video_path

'../../dataset_aihub/'

In [6]:
print(os.listdir(video_path))

['abnormal_강동욱', 'normal_강동욱', '.DS_Store', 'abnormal_김성욱', 'normal_김성욱', 'normal_강민지', 'abnormal_김성욱.pickle', 'normal_강민지.pickle', 'abnormal_강민지', 'normal_강동욱.pickle', 'normal_김성욱.pickle', 'abnormal_강민지.pickle']


In [7]:
# 영상 데이터에서 mp pose landmark dot 데이터 추출 부분
# raw_data = []

In [8]:
# for fold in os.listdir(video_path):
#    for video_name in os.listdir(video_path + '/' + fold):
#        if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
#            if video_name.split('_')[1] == 'normal': label = 0
#            else: label = 1
#            skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
#            if skel_data_n != False:
#                seq_list_n = skel_data_n[:30]
#                seq_list_f = skel_data_f[:30]
#                raw_data.append({'key':label, 'value':seq_list_n})
#                raw_data.append({'key':label, 'value':seq_list_f})
# random.shuffle(raw_data)

In [9]:
fold = 'abnormal_김성욱'
raw_data = []

In [10]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713925506.950551 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1713925511.373322 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925514.912259 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925518.465557 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925522.637580 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925526.109154 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925529.713653 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925533.949761 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713925537.484679 2150928 gl_context.cc:357] GL vers

In [11]:
import pickle

In [12]:
filename = video_path + fold + '.pickle'
print(filename)

../../dataset_aihub/abnormal_김성욱.pickle


In [13]:
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

In [15]:
fold = 'normal_김성욱'
raw_data = []

In [16]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713923467.303726 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923472.308378 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923475.638770 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923478.803307 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923482.759004 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923487.288174 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923490.541359 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923494.509557 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713923498.627024 2117701 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:0

In [20]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/normal_김성욱.pickle


In [14]:
fold = 'normal_강민지'
raw_data = []

In [15]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713926004.397779 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926008.272915 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926011.759046 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926015.482457 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926019.153446 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926023.855109 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926026.943515 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926030.758632 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926034.335859 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
Context le

In [16]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/normal_강민지.pickle


In [17]:
fold = 'abnormal_강민지'
raw_data = []

In [18]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713926420.539487 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926424.109958 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
Context leak detected, msgtracer returned -1
I0000 00:00:1713926427.611580 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926431.016424 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926434.419624 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926437.867506 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926441.324933 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926444.885199 2150928 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713926448.132161 2150928 gl_context.cc:357] GL version: 2.1 (2

In [19]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/abnormal_강민지.pickle


In [9]:
fold = 'normal_강동욱'
raw_data = []

In [13]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713927269.082791 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927273.199822 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927276.546000 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927279.488086 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927282.855879 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927286.539801 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927290.546852 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927294.461070 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713927297.843639 2165579 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:0

In [16]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/normal_강동욱.pickle


In [8]:
fold = 'abnormal_강동욱'
raw_data = []

In [9]:
for video_name in os.listdir(video_path + '/' + fold):
    if int(video_name.split('_')[2][:2]) >= 30: # video name 참조
        if video_name.split('_')[1] == 'normal': label = 0
        else: label = 1
        skel_data_n, skel_data_f = get_skeleton('{}/{}'.format(video_path + '/' + fold, video_name), attention_dot, draw_line)
        if skel_data_n != False:
            seq_list_n = skel_data_n[:30]
            seq_list_f = skel_data_f[:30]
            raw_data.append({'key':label, 'value':seq_list_n})
            raw_data.append({'key':label, 'value':seq_list_f})

I0000 00:00:1713929276.202884   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1713929280.671034   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929284.409014   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929287.912746   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929291.369299   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929294.468276   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929298.430020   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929301.655776   12242 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M2
I0000 00:00:1713929304.937523   12242 gl_context.cc:357] GL vers

In [12]:
filename = video_path + fold + '.pickle'
print(filename)
with open(filename, 'wb') as f:
    pickle.dump(raw_data, f)

../../dataset_aihub/abnormal_강동욱.pickle


In [13]:
import glob
pickles = sorted(glob.glob('../../dataset_aihub/*.pickle'))

In [14]:
print(pickles)

['../../dataset_aihub/abnormal_강동욱.pickle', '../../dataset_aihub/abnormal_강민지.pickle', '../../dataset_aihub/abnormal_김성욱.pickle', '../../dataset_aihub/normal_강동욱.pickle', '../../dataset_aihub/normal_강민지.pickle', '../../dataset_aihub/normal_김성욱.pickle']


In [16]:
raw_data = []
for onepickle in pickles:
    with open(onepickle, 'rb') as f:
        raw_data += pickle.load(f)

In [17]:
len(raw_data)

1384

In [31]:
for onepickle in pickles:
    with open(onepickle, 'rb') as f:
        data = pickle.load(f)
        print(onepickle,len(data))

../../dataset_aihub/abnormal_강동욱.pickle 170
../../dataset_aihub/abnormal_강민지.pickle 362
../../dataset_aihub/abnormal_김성욱.pickle 142
../../dataset_aihub/normal_강동욱.pickle 164
../../dataset_aihub/normal_강민지.pickle 250
../../dataset_aihub/normal_김성욱.pickle 296


In [27]:
len(data)

164

In [40]:
# dataset 길이 출력

nd = 0
ad = 0
for i in range(len(raw_data)):
    if raw_data[i]['key'] == 0:
        nd += 1
    else:
        ad += 1
print('normal data:', nd, '| abnormal data:', ad)

normal data: 14 | abnormal data: 16


In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

- 모델링

In [42]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])

    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [43]:
split_ratio = [0.8, 0.1, 0.1]
train_len = int(len(raw_data) * split_ratio[0])
val_len = int(len(raw_data) * split_ratio[1])
test_len = len(raw_data) - train_len - val_len

print('{}, {}, {}'.format(train_len, val_len, test_len))

24, 3, 3


In [44]:
train_dataset = MyDataset(raw_data)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [45]:
# LSTM
class skeleton_LSTM(nn.Module):
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x

In [46]:
# 모델 초기화

def init_model():
    global net, loss_fn, optim
    plt.rc('font', size = 10)
    net = skeleton_LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net.parameters(), lr=0.0001)

# epoch 카운터 초기화
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

# 모든 Log를 초기화
def init_log():
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    plt.rc('font', size = 10)
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

In [47]:
def record_train_log(_tloss, _tacc, _time):
    # Train Log 기록
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)

def record_valid_log(_vloss, _vacc):
    # Validation Log 기록
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

def last(log_list):
    # last 안의 마지막 숫자를 반환(print_log 함수에서 사용)
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

def print_log():
    # 학습 추이 출력 : 소숫점 3자리까지
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3} | T_Loss {:5} | T_Acc {:5} | V_Loss {:5} | V_Acc {:5} | {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)

    log_stack.append(log_str)
    
    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99)
    hist_fig.patch.set_facecolor('white')

    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train_Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid_Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')

    # Acc, Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train_Acc', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid_Acc', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')

    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines])
    loss_axis.grid()
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()

    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])

In [48]:
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

# 학습 알고리즘
def epoch(data_loader, mode = 'train'):
    global epoch_cnt

    # 사용되는 변수 초기화
    iter_loss, iter_acc, last_grad_performed = [], [], False

    # 1 iteration 학습 알고리즘(for문을 나오면 1 epoch 완료)
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        # 1. Feed-forward
        if mode == 'train':
            net.train()
        else:
            # 학습때만 쓰이는 Dropout, Batch Mormalization을 미사용
            net.eval()

        result = net(data) # 1 Batch에 대한 결과가 모든 Class에 대한 확률값으로
        _, out = torch.max(result, 1) # result에서 최대 확률값을 기준으로 예측 class 도출( _ : 값 부분은 필요 없음, out : index 중 가장 큰 하나의 데이터)

        # 2. Loss 계산
        loss = loss_fn(result, label) # GT 와 Label 비교하여 Loss 산정
        iter_loss.append(loss.item()) # 학습 추이를 위하여 Loss를 기록

        # 3. 역전파 학습 후 Gradient Descent
        if mode == 'train':
            optim.zero_grad() # 미분을 통해 얻은 기울기를 초기화 for 다음 epoch
            loss.backward() # 역전파 학습
            optim.step() # Gradient Descent 수행
            last_grad_performed = True # for문을 나가면 epoch 카운터 += 1

        # 4. 정확도 계산
        acc_partial = (out == label).float().sum() # GT == Label 인 개수
        acc_partial = acc_partial / len(label) # ( TP / (TP + TM)) 해서 정확도 산출
        iter_acc.append(acc_partial.item()) # 학습 추이를 위하여 Acc. 기록

    # 역전파 학습 후 Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1

    clear_memory()

    # loss와 acc의 평균값 for 학습추이 그래프, 모든 GT와 Label 값 for 컨퓨전 매트릭스
    return np.average(iter_loss), np.average(iter_acc)

def epoch_not_finished():
    # 에폭이 끝남을 알림
    return epoch_cnt < maximum_epoch

- 학습 진행

In [49]:
# Training initialization
init_model()
init_epoch()
init_log()
maximum_epoch = EPOCH

# Training iteration

while epoch_not_finished():
    start_time = time.time()

    tloss, tacc = epoch(train_loader, mode = 'train')

    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)

    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode = 'val')
        record_valid_log(vloss, vacc)

    print_log()

print('\n Training completed!')

Epoch: 700 | T_Loss  0.69 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.511
Epoch: 699 | T_Loss 0.691 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.504
Epoch: 698 | T_Loss  0.69 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.507
Epoch: 697 | T_Loss 0.691 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.506
Epoch: 696 | T_Loss 0.688 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.501
Epoch: 695 | T_Loss 0.691 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 |   1.5
Epoch: 694 | T_Loss 0.692 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.495
Epoch: 693 | T_Loss 0.691 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.494
Epoch: 692 | T_Loss 0.689 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.491
Epoch: 691 | T_Loss 0.694 | T_Acc 0.542 | V_Loss 0.614 | V_Acc   1.0 | 1.489
Epoch: 690 | T_Loss 0.689 | T_Acc 0.542 | V_Loss 0.613 | V_Acc   1.0 | 1.493
Epoch: 689 | T_Loss 0.695 | T_Acc 0.542 | V_Loss 0.613 | V_Acc   1.0 | 1.488
Epoch: 688 | T_Loss 0.688 | T_Acc 0.542 | V_Loss 0.613 | V_Acc   1.0 | 1.488

In [50]:
# 정확도 검증
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode = 'test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc.: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))

Test Acc.: 0.0
Test Loss: 0.7791


- 1분 원본 영상으로 모델 테스트 하기

In [51]:
# 영상 resize 및 추출
test_video_name = 'C_3_12_43_BU_SMC_10-14_12-17-14_CC_RGB_DF2_F2'
#test_video_path = f'/content/drive/MyDrive/Colab_Notebooks/Anomaly Detection/{test_video_name}.mp4'
test_video_path = f'/home/raccoon/jnwork/kimsungwook/zerobase_DL_project/source_code/model/{test_video_name}.mp4'
cv2.destroyAllWindows()
cap = cv2.VideoCapture(test_video_path)
img_list = []

if cap.isOpened():

    while True:
        ret, img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            img_list.append(img)
            # cv2_imshow(img)
            # cv2.waitKey(1)
        else:
            break

cap.release()
cv2.destroyAllWindows()

print('저장된 frame의 개수: {}'.format(len(img_list)))

저장된 frame의 개수: 181


In [52]:
"""Yolov5 + Mediapipe Version"""

net.eval()

length = 30 # frame 상태를 표시할 길이
out_img_list = []
dataset = []
status = 'None'
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
print('시퀀스 데이터 분석 중...')

xy_list_list = []
for img in tqdm(img_list):
    res = yolo_model(img)
    res_refine = res.pandas().xyxy[0].values

    nms_human = len(res_refine)
    if nms_human > 0:
        for bbox in res_refine:
            xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
            if xx1 < 0:
                xx1 = 0
            elif xx2 > 639:
                xx2 = 639
            if yy1 < 0:
                yy1 = 0
            elif yy2 > 639:
                yy2 = 639

            start_point = (xx1, yy1)
            end_point = (xx2, yy2)
            if bbox[4] > y_CONFIDENCE:
                img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2)

                c_img = img[yy1:yy2, xx1:xx2]
                results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolo 바운딩 box 안에서 landmark dot 추출
                if not results.pose_landmarks: continue
                xy_list = []
                idx = 0
                draw_line_dic = {}
                for x_and_y in results.pose_landmarks.landmark:
                    if idx in attention_dot:
                        xy_list.append(x_and_y.x)
                        xy_list.append(x_and_y.y)
                        x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                        draw_line_dic[idx] = [x, y]
                    idx += 1

                xy_list_list.append(xy_list)
                for line in draw_line:
                    x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                    x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                    c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 3)

                if len(xy_list_list) == length:
                    dataset = []
                    dataset.append({'key' : 0, 'value' : xy_list_list})
                    dataset = MyDataset(dataset)
                    dataset = DataLoader(dataset)
                    xy_list_list = []

                    for data, label in dataset:
                        data = data.to(device)
                        with torch.no_grad():
                            result = net(data)
                            _, out = torch.max(result, 1)
                            if out.item() == 0: status = 'Normal'
                            else: status = 'Theft'

    cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 2)
    out_img_list.append(img)

I0000 00:00:1713501158.226075    3656 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713501158.249397    6889 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.161.07), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2


시퀀스 데이터 분석 중...


100%|██████████████████████████████████████████████████| 181/181 [00:13<00:00, 13.84it/s]


In [53]:
# 테스트 원본 영상 내보내기
filename = './output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 3
frameSize = (640, 640)
isColor = True
out = cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor)
for out_img in out_img_list:
    out.write(out_img)
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


- 모델 저장

In [55]:
# 모델 저장하기
PATH = './'
model_name = 'LSTM.pt'
torch.save(net.state_dict(), PATH + model_name)

In [57]:
# 영상 resize 및 추출
test_video_name = 'sample_1.mov'
#test_video_path = f'/content/drive/MyDrive/Colab_Notebooks/Anomaly Detection/{test_video_name}'
test_video_path = f'/home/raccoon/jnwork/kimsungwook/zerobase_DL_project/source_code/model/{test_video_name}'
cv2.destroyAllWindows()
cap = cv2.VideoCapture(test_video_path)
img_list = []

if cap.isOpened():

    while True:
        ret, img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            img_list.append(img)
            # cv2_imshow(img)
            # cv2.waitKey(1)
        else:
            break

cap.release()
cv2.destroyAllWindows()

print('저장된 frame의 개수: {}'.format(len(img_list)))

저장된 frame의 개수: 1483


In [58]:
"""Yolov5 + Mediapipe Version"""

net.eval()

length = 30 # frame 상태를 표시할 길이
out_img_list = []
dataset = []
status = 'None'
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
print('시퀀스 데이터 분석 중...')

xy_list_list = []
for img in tqdm(img_list):
    res = yolo_model(img)
    res_refine = res.pandas().xyxy[0].values

    nms_human = len(res_refine)
    if nms_human > 0:
        for bbox in res_refine:
            xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
            if xx1 < 0:
                xx1 = 0
            elif xx2 > 639:
                xx2 = 639
            if yy1 < 0:
                yy1 = 0
            elif yy2 > 639:
                yy2 = 639

            start_point = (xx1, yy1)
            end_point = (xx2, yy2)
            if bbox[4] > y_CONFIDENCE:
                img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2)

                c_img = img[yy1:yy2, xx1:xx2]
                results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolo 바운딩 box 안에서 landmark dot 추출
                if not results.pose_landmarks: continue
                xy_list = []
                idx = 0
                draw_line_dic = {}
                for x_and_y in results.pose_landmarks.landmark:
                    if idx in attention_dot:
                        xy_list.append(x_and_y.x)
                        xy_list.append(x_and_y.y)
                        x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                        draw_line_dic[idx] = [x, y]
                    idx += 1

                xy_list_list.append(xy_list)
                for line in draw_line:
                    x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                    x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                    c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 3)

                if len(xy_list_list) == length:
                    dataset = []
                    dataset.append({'key' : 0, 'value' : xy_list_list})
                    dataset = MyDataset(dataset)
                    dataset = DataLoader(dataset)
                    xy_list_list = []

                    for data, label in dataset:
                        data = data.to(device)
                        with torch.no_grad():
                            result = net(data)
                            _, out = torch.max(result, 1)
                            if out.item() == 0: status = 'Normal'
                            else: status = 'Theft'

    cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 2)
    out_img_list.append(img)

I0000 00:00:1713506886.337675    3656 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1713506886.362469   11895 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.161.07), renderer: NVIDIA GeForce GTX 1080 Ti/PCIe/SSE2


시퀀스 데이터 분석 중...


100%|████████████████████████████████████████████████| 1483/1483 [01:31<00:00, 16.28it/s]


In [59]:
# 테스트 원본 영상 내보내기
filename = './output2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 3
frameSize = (640, 640)
isColor = True
out = cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor)
for out_img in out_img_list:
    out.write(out_img)
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
